## 作業
嘗試比較用 color histogram 和 HOG 特徵來訓練的 SVM 分類器在 cifar10 training 和 testing data 上準確度的差別

In [1]:
import os
import keras
os.environ["CUDA_VISIBLE_DEVICES"] = "" # 使用 CPU

import numpy as np
import cv2 # 載入 cv2 套件
import matplotlib.pyplot as plt

train, test = keras.datasets.cifar10.load_data()

Using TensorFlow backend.


In [2]:
x_train, y_train = train
x_test, y_test = test
y_train = y_train.astype(int)
y_test = y_test.astype(int)

#### 產生直方圖特徵的訓練資料

In [3]:
x_train_histogram = []
x_test_histogram = []

# 對於所有訓練資料
for i in range(len(x_train)):
    chans = cv2.split(x_train[i]) # 把圖像的 3 個 channel 切分出來
    # 對於所有 channel
    hist_feature = []
    for chan in chans:
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) # 切成 16 個 bin
        hist_feature.extend(hist.flatten())
    # 把計算的直方圖特徵收集起來
    x_train_histogram.append(hist_feature)

# 對於所有測試資料也做一樣的處理
for i in range(len(x_test)):
    chans = cv2.split(x_test[i]) # 把圖像的 3 個 channel 切分出來
    # 對於所有 channel
    hist_feature = []
    for chan in chans:
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) # 切成 16 個 bin
        hist_feature.extend(hist.flatten())
    x_test_histogram.append(hist_feature)

x_train_histogram = np.array(x_train_histogram)
x_test_histogram = np.array(x_test_histogram)


#### 產生 HOG 特徵的訓練資料
* HOG 特徵通過計算和統計圖像局部區域的梯度方向直方圖來構建特徵，具體細節不在我們涵蓋的範圍裡面，有興趣的同學請參考[補充資料](https://www.cnblogs.com/zyly/p/9651261.html)哦

In [4]:
# SZ=20
bin_n = 16 # Number of bins

def hog(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    bins = np.int32(bin_n*ang/(2*np.pi))    # quantizing binvalues in (0...16)
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)     # hist is a 64 bit vector
    return hist.astype(np.float32)

x_train_hog = np.array([hog(x) for x in x_train])
x_test_hog = np.array([hog(x) for x in x_test])

#### SVM model
* SVM 是機器學習中一個經典的分類算法，具體細節有興趣可以參考 [該知乎上的解釋](https://www.zhihu.com/question/21094489)，我們這裡直接調用 opencv 中實現好的函數

#### 用 histogram 特徵訓練 SVM 模型
* 訓練過程可能會花點時間，請等他一下

In [5]:
SVM_hist = cv2.ml.SVM_create()
SVM_hist.setKernel(cv2.ml.SVM_LINEAR)
SVM_hist.setGamma(5.383)
SVM_hist.setType(cv2.ml.SVM_C_SVC)
SVM_hist.setC(2.67)

#training
SVM_hist.train(x_train_histogram, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hist_train = SVM_hist.predict(x_train_histogram)
_, y_hist_test = SVM_hist.predict(x_test_histogram)

#### 用 HOG 特徵訓練 SVM 模型
* 訓練過程可能會花點時間，請等他一下

In [6]:
SVM_hog = cv2.ml.SVM_create()
SVM_hog.setKernel(cv2.ml.SVM_LINEAR)
SVM_hog.setGamma(5.383)
SVM_hog.setType(cv2.ml.SVM_C_SVC)
SVM_hog.setC(2.67)

#training
SVM_hog.train(x_train_hog, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hog_train = SVM_hog.predict(x_train_hog)
_, y_hog_test = SVM_hog.predict(x_test_hog)

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix


acc_train_chist = accuracy_score(y_train, y_hist_train)
acc_train_hog = accuracy_score(y_train, y_hog_train)
acc_test_chist = accuracy_score(y_test, y_hist_test)
acc_test_hog = accuracy_score(y_test, y_hog_test)

train_chist_matrix = confusion_matrix(y_train, y_hist_train)
test_chist_matrix = confusion_matrix(y_test, y_hist_test)
train_hog_matrix = confusion_matrix(y_train, y_hog_train)
test_hog_matrix = confusion_matrix(y_test, y_hog_test)


In [10]:
print('train_accuracy: color_hist: {} hog: {}'.format(acc_train_chist, acc_train_hog))
print('test_accuracy:  color_hist: {} hog: {}'.format(acc_test_chist, acc_test_hog))

train_accuracy: color_hist: 0.14846 hog: 0.19686
test_accuracy:  color_hist: 0.1448 hog: 0.2006


In [15]:
import pandas as pd
from IPython.display import display
display(pd.DataFrame(train_chist_matrix), pd.DataFrame(test_chist_matrix))
display(pd.DataFrame(train_hog_matrix), pd.DataFrame(test_hog_matrix))

,0,1,2,3,4,5,6,7,8,9
0,1957,279,1272,344,272,265,29,71,453,58
1,894,225,1128,334,325,1229,16,49,654,146
2,834,167,1215,619,1143,590,48,60,241,83
3,952,152,999,549,603,1049,34,59,493,110
4,530,87,1162,645,1587,658,41,48,192,50
5,990,102,1029,607,642,1024,27,49,442,88
6,480,142,1014,597,1355,1005,85,99,146,77
7,714,132,1500,550,848,753,14,44,363,82
8,1929,113,1345,306,170,366,5,28,664,74
9,1018,226,1664,281,247,768,7,18,698,73


,0,1,2,3,4,5,6,7,8,9
0,383,67,255,60,43,58,6,20,99,9
1,174,50,221,61,67,248,3,7,135,34
2,189,35,232,116,225,118,16,15,41,13
3,198,25,199,103,130,221,7,10,79,28
4,97,15,237,130,327,127,10,10,40,7
5,203,16,230,120,117,194,8,9,92,11
6,95,24,210,113,292,190,18,25,26,7
7,165,30,284,107,167,148,3,12,66,18
8,382,18,281,72,34,76,0,3,119,15
9,213,50,345,55,47,153,1,2,124,10


,0,1,2,3,4,5,6,7,8,9
0,277,1164,658,171,410,230,53,1268,183,586
1,153,2158,54,237,163,47,80,836,112,1160
2,158,324,1250,624,399,506,101,1308,48,282
3,201,335,671,1239,270,537,107,1270,31,339
4,117,298,828,705,461,510,60,1688,22,311
5,131,236,789,1087,231,634,86,1542,14,250
6,243,330,522,782,195,715,172,1619,56,366
7,101,470,409,1040,211,263,83,1935,23,465
8,54,1506,461,73,505,104,22,1209,255,811
9,103,1431,155,425,122,54,39,1136,73,1462


,0,1,2,3,4,5,6,7,8,9
0,64,215,138,37,93,41,11,251,36,114
1,22,442,12,40,37,7,25,154,25,236
2,30,61,258,136,80,108,11,244,12,60
3,39,69,116,239,60,107,32,255,5,78
4,15,53,173,135,86,111,15,339,5,68
5,27,48,158,226,40,125,32,298,3,43
6,37,65,116,165,24,148,28,337,12,68
7,25,94,77,219,50,53,12,382,6,82
8,15,289,95,14,91,22,3,259,54,158
9,28,260,38,73,27,16,7,209,14,328
